# Human & Object Detection using YOLOv8

## Objective
This notebook demonstrates:
- Object and human detection using a **pre-trained YOLOv8 model**
- Drawing bounding boxes using **OpenCV**
- (Optional Advanced) **Basic posture classification** of  like standing, sitting and bending.

## Required Libraries

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

## Load YOLOv8 Pre-trained Model

In [7]:
 
model = YOLO("yolov8n.pt")
print("Model loaded successfully!")


Model loaded successfully!


## Posture Classification Logic

In [11]:
def classify_posture(box):
    """
    [Tried a simple logic]
    Classify posture based on bounding box aspect ratio.
    
    Parameters:
    box (tuple): (x1, y1, x2, y2)
    
    Returns:
    str: Standing / Sitting / Bending
    """
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1

    ratio = height / width

    if ratio > 1.8:
        return "Standing"
    elif 1.2 < ratio <= 1.8:
        return "Sitting"
    else:
        return "Bending"


## Object Detection (On Video)

In [14]:
video_path = r"81496-576306063_small.mp4"  
output_path = r"output_detected.mp4"

cap = cv2.VideoCapture(video_path)

# video properties 
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 25  # fallback

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    for r in results:
        for box, cls, conf in zip(r.boxes.xyxy, r.boxes.cls, r.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            label = model.names[int(cls)]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"{label} {conf:.2f}"

            if label == "person":
                posture = classify_posture((x1, y1, x2, y2))
                text += f", {posture}"

            cv2.putText(frame, text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print("Saved:", output_path)

Saved: output_detected.mp4
